In [1]:
import numpy as np
import pandas as pd
import time
from datetime import date, datetime
import os

os.environ["config_path"] = "/home/asingal/home/SeQUeNCe/example/trajectree_integration/MIM_config.json"

from sequence.topology.router_net_topo import RouterNetTopo
from sequence.app.request_app import RequestApp
import sequence.utils.log as log


/home/asingal/home/.venv/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/asingal/home/.venv/lib/python3.10/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


Using Trajectree backend for entanglement generation


In [2]:
today = date.today()

CONFIG_FILE = "MIM_config.json"

# meta params
NO_TRIALS = 1
OUTPUT_FILE = "results/var_mem_"+str(today)+".csv"
LOGGING = False
LOG_OUTPUT = "results/var_mem_log.csv"
MODULE_TO_LOG = ["timeline", "memory", "bsm", "generation", "request_app"]

# simulation params
PREP_TIME = int(1e12)  # 1 second
COLLECT_TIME = int(1e12)  # 10 seconds


# qc params
# QC_FREQ = 1e11
QC_FREQ = 1e11



# application params
APP_NODE_NAME = "left"
OTHER_NODE_NAME = "right"
num_memories = [4,8,16,32,64]  # iterate through these


# storing data
data_dict = {"Memory Count": [],
             "Average Throughput": [],
             "Std. Throughput": [],
             "Average TTT": [],
             "Std. TTT":[],
             }


tick = time.time()

collect_time_base = 3e12

for i, num_memo in enumerate(num_memories):
    print(f"Running {NO_TRIALS} trials for memory count {num_memo} ({i + 1}/{len(num_memories)})")
    data_dict["Memory Count"].append(num_memo)
    throughputs = np.zeros(NO_TRIALS)
    time_to_thousand = np.zeros(NO_TRIALS)

    COLLECT_TIME = collect_time_base/num_memo

    for trial_no in range(NO_TRIALS):
        # establish network
        net_topo = RouterNetTopo(CONFIG_FILE)

        # timeline setup
        tl = net_topo.get_timeline()
        tl.stop_time = PREP_TIME + COLLECT_TIME

        if LOGGING:
            # set log
            if num_memo == num_memories[0]:
                log.set_logger(__name__, tl, LOG_OUTPUT)
                log.set_logger_level('WARN')
                for module in MODULE_TO_LOG:
                    log.track_module(module)
            elif num_memo == num_memories[1]:
                for module in MODULE_TO_LOG:
                    log.remove_module(module)

        # network configuration
        routers = net_topo.get_nodes_by_type(RouterNetTopo.QUANTUM_ROUTER)
        bsm_nodes = net_topo.get_nodes_by_type(RouterNetTopo.BSM_NODE)

        for j, node in enumerate(routers + bsm_nodes):
            node.set_seed(int(datetime.now().timestamp()) + j + (trial_no * 3))

        # set quantum channel parameters
        for qc in net_topo.get_qchannels():
            qc.frequency = QC_FREQ

        # establish app on left node
        start_node = None
        for node in routers:
            if node.name == APP_NODE_NAME:
                start_node = node
                break
        if not start_node:
            raise ValueError(f"Invalid app node name {APP_NODE_NAME}")
        end_node = None
        for node in routers:
            if node.name == OTHER_NODE_NAME:
                end_node = node
                break
        if not start_node:
            raise ValueError(f"Invalid other node name {OTHER_NODE_NAME}")

        app_start = RequestApp(start_node)
        app_end = RequestApp(end_node)

        # initialize and start app
        tl.init()
        app_start.start(OTHER_NODE_NAME, PREP_TIME, PREP_TIME + COLLECT_TIME, num_memo, 1.0)
        tl.run()

        # Used for debugging
        # attempt = app_start.node.total_attempts
        # success = app_start.node.succesful_attempts
        # prob = success/attempt

        throughputs[trial_no] = app_start.get_throughput()        
        

        print(f"\tCompleted trial {trial_no + 1}/{NO_TRIALS}")



    print("Finished trials.")

    avg_throughput = np.mean(throughputs)
    std_throughput = np.std(throughputs)
    avg_TTT = np.mean(time_to_thousand)*1e-12
    std_TTT = np.std(time_to_thousand)*1e-12
    
    print(f"Average throughput: {avg_throughput} +/- {std_throughput}")
    print(f"time to thousand entanglements:", time_to_thousand)

    data_dict["Average Throughput"].append(avg_throughput)
    data_dict["Std. Throughput"].append(std_throughput)
    data_dict["Average TTT"].append(avg_TTT)
    data_dict["Std. TTT"].append(std_TTT)
             

df = pd.DataFrame(data_dict)
df.to_csv(OUTPUT_FILE, index=False)

runtime = time.time() - tick
print("Total runtime:", runtime)


Running 1 trials for memory count 4 (1/5)
calling trajectree ent generation
checking for cached entanglement
cached entanglement not found, running simulation
trajectory_probs [9.32911070e-01 1.08776651e-03 1.31212080e-06]
trajectory_probs [8.70323064e-01 1.01478942e-03 1.22409202e-06]
trajectory_probs [8.32336726e-01 1.83588077e-02 4.22388462e-04 1.83588055e-02
 4.04876741e-04 9.29243542e-06 4.22399089e-04 9.28461007e-06
 2.13249546e-07]
trajectory_probs [7.96078441e-01 1.75237217e-02 4.03629399e-04 1.75237215e-02
 3.85591928e-04 8.93961207e-06 4.03605472e-04 8.91854173e-06
 1.53955762e-07]
trajectory_probs [7.94488634e-01 7.93712409e-04 7.92884585e-07 7.93712409e-04
 7.92936941e-07 7.92109927e-10 7.92884585e-07 7.92109927e-10
 7.91283774e-13]
trajectory_probs [7.92902002e-01 7.92127327e-04 7.91301157e-07 7.92127327e-04
 7.91353408e-07 7.90528046e-10 7.91301157e-07 7.90528046e-10
 7.89703543e-13]
trajectory_probs [1.74887258e-02 1.74716347e-05 1.74534209e-08 1.74716785e-05
 1.74546041

/home/asingal/home/SeQUeNCe/example/trajectree_integration/plugins/generation_trajectree.py:249: ComplexWarning: Casting complex values to real discards the imaginary part
  num_attempts = np.random.geometric(success_probability)  # time to first occurence in a Bernoulli process is a geometric distribution.


trajectory_probs [1.75237216e-02 3.85635091e-04 8.89873541e-06 4.21525490e-04
 9.35862787e-06 2.30719783e-07 9.27760146e-06 1.55097311e-07
 2.41746466e-09]
trajectory_probs [4.20683684e-04 4.20272673e-07 4.19834333e-10 4.20272851e-07
 4.19862242e-10 4.19424330e-13 4.19834155e-10 4.19423974e-13
 4.18986519e-16]
trajectory_probs [4.19843549e-04 4.19433436e-07 4.18995818e-10 4.19442587e-07
 4.19032865e-10 4.18595666e-13 4.18986676e-10 4.18577400e-13
 4.18140674e-16]
trajectory_probs [4.02823337e-04 4.02425624e-07 4.02014194e-10 4.02428545e-07
 4.02031222e-10 4.01620195e-13 4.02011276e-10 4.01614365e-13
 4.01203764e-16]
trajectory_probs [4.02018889e-04 4.01617271e-07 4.01216055e-10 4.01626166e-07
 4.01224941e-10 4.00824117e-13 4.01207169e-10 4.00806363e-13
 4.00405957e-16]
trajectory_probs [1.74887258e-02 1.74716347e-05 1.74534209e-08 1.74716785e-05
 1.74546041e-08 1.74364080e-11 1.74533771e-08 1.74363206e-11
 1.74181436e-14]
trajectory_probs [1.74537995e-02 1.74367468e-05 1.74185610e-08 1

/home/asingal/home/quimb/quimb/tensor/tensor_1d.py:1867: RuntimeWarning: divide by zero encountered in divide
  self[insert].modify(data=self[insert].data / norm**0.5)
/home/asingal/home/quimb/quimb/tensor/tensor_1d.py:1867: RuntimeWarning: invalid value encountered in divide
  self[insert].modify(data=self[insert].data / norm**0.5)


trajectory_probs [1.74887255e-02 1.74716365e-05 1.74534185e-08 1.74720666e-05
 1.74549939e-08 1.74367932e-11 1.74529889e-08 1.74359349e-11
 1.74177540e-14]
trajectory_probs [1.74537996e-02 1.74367469e-05 1.74185610e-08 1.74367826e-05
 1.74197466e-08 1.74015784e-11 1.74185253e-08 1.74015071e-11
 1.73833580e-14]
trajectory_probs [1.75237216e-02 4.21578225e-04 9.27988518e-06 3.85551946e-04
 9.38985505e-06 1.04883754e-07 8.95971968e-06 2.19005805e-07
 2.45881754e-09]
trajectory_probs [1.74887255e-02 1.74720590e-05 1.74529965e-08 1.74716344e-05
 1.74549843e-08 1.74359403e-11 1.74534206e-08 1.74367878e-11
 1.74177637e-14]
trajectory_probs [1.74537996e-02 1.74367827e-05 1.74185253e-08 1.74367469e-05
 1.74197466e-08 1.74015070e-11 1.74185611e-08 1.74015786e-11
 1.73833580e-14]
trajectory_probs [1.74887259e-02 1.74716787e-05 1.74533773e-08 1.74716371e-05
 1.74546065e-08 1.74363230e-11 1.74534189e-08 1.74364060e-11
 1.74181416e-14]
trajectory_probs [1.74537997e-02 1.74371666e-05 1.74181420e-08 1

AssertionError: invalid event time for process scheduled on right